In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/Shareddrives/CS229Buddies/project')

In [4]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import collections
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve, f1_score, plot_precision_recall_curve
from scipy.stats import pearsonr
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import operator
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


In [36]:
df=pd.read_csv("data/GO_matrix")
df=df.T
df.columns=df.iloc[0,:]
df=df.iloc[1:,:]

In [14]:
# df.set_index("Term|NES")

,101_19yr_Female_Caucasian,102_19yr_Male_Caucasian,103_19yr_Male_Caucasian,104_19yr_Male_Caucasian,105_20yr_Male_Caucasian,106_20yr_Female_Caucasian,107_31yr_Female_Caucasian,108_31_female_Caucasian,109_32_male_Unknown,110_32_female_BlackPuertoRican,111_33yr_Male_Caucasian,112_33yr_Male_Caucasian,114_37yr_Female_Unknown,115_37yr_Female_Unknown,117_37yr_Male_Caucasian,118_39yr_Male_Caucasian,120_51yr_Male_Caucasian,121_52yr_Male_Caucasian,122_55yr_Male_Caucasian,123_57_male_Caucasian,124_60yr_Male_Caucasian,125_71yr_Female_Caucasian,126_75_male_Caucasian,127_78yr_Male_Caucasian,128_80yr_Female_Caucasian,129_94_Male_Caucasian,130_89_Male_Caucasian,131_87_Male_Caucasian,132_90_Male_Caucasian,133_89_Male_Caucasian,134_86_Male_Caucasian,135_92_Male_Caucasian,136_87_Male_Caucasian,14_1yr_Male_Asian,15_12yr_Male_Caucasian,155_22yr_Male_Caucasian,156_25yr_Female_Caucasian,157_29yr_Male_Unknown,158_24YR_Male_Caucasian,159_23YR_Male_AmericanIndian,...,69_87_Female_caucasian,70_87_Male_caucasian,71_87_Male_caucasian,72_87_Male_Black,73_88_Male_Caucasian,74_89_Male_Caucasian,75_90_Male_Caucasian,76_91_Male_Caucasian,77_92_Female_Caucasian,78_96_Male_Caucasian,79_1_male_Caucasian,80_2_female_Caucasian,81_3_male_LatinoHispanic,82_3_male_Unknown,83_5yr_Male_Black,84_6yr_Male_Black,85_7yr_Male_Black,86_7yr_Male_Caucasian,87_8yr_Male_Caucasian,88_8yr_Male_Caucasian,89_9yr_Female_Black,90_10yr_Male_Black,91_10yr_Male_Caucasian,92_11_female_Caucasian,93_11_female_Caucasian,94_12_male_Caucasian,95_13yr_Male_WhiTEEASTINDIAN,97_16yr_Female_Unknown,98_17yr_Male_Caucasian,99_17yr_Male_Caucasian,139_8_Female-HGPS_Caucasian,141_8_Male-HGPS_Unknown,197_2yr3mos_F_Unknown,198_3yr9mos_F_Unknown,199_4yr8mos_F_Unknown,200_8yr6mos_M_Unknown,201_6ys11mos_F_Unknown,202_5yr0mos_F_Unknown,203_8yr10mos_M_Unknown,204_3yr0mos_F_Unknown
Term|NES,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'de novo' posttranslational protein folding (GO:0051084),0.350890,0.347854,0.323341,0.319741,0.348308,0.353414,0.350498,0.360435,0.360693,0.334585,0.331541,0.350204,0.354583,0.326394,0.327429,0.344968,0.331023,0.345603,0.340241,0.338566,0.341722,0.316734,0.353803,0.345665,0.354654,0.347367,0.344354,0.352872,0.347037,0.352841,0.338798,0.357623,0.358612,0.335348,0.332923,0.340830,0.338460,0.350980,0.324259,0.329552,...,0.334397,0.329551,0.330741,0.372936,0.319973,0.357746,0.393503,0.347189,0.367100,0.369298,0.335381,0.348068,0.343269,0.329249,0.337968,0.354243,0.352401,0.335017,0.328180,0.350953,0.342896,0.333524,0.348205,0.323334,0.321925,0.333707,0.344194,0.355421,0.327752,0.353976,0.339281,0.333005,0.351428,0.353690,0.335838,0.333286,0.345568,0.338217,0.351784,0.331427
3'-phosphoadenosine 5'-phosphosulfate metabolic process (GO:0050427),0.055188,0.045778,0.031528,0.066636,0.055484,0.070795,0.040182,0.052664,0.060449,0.055535,0.072680,0.034272,0.037635,0.064548,0.053665,0.075708,0.057642,0.047385,0.055292,0.048208,0.060454,0.085034,0.030776,0.077477,0.073350,0.057372,0.048812,0.094054,0.062942,0.050739,0.068135,0.086050,0.077861,0.046128,0.049494,0.045037,0.063819,0.049119,0.072949,0.071956,...,0.057631,0.055631,0.059293,0.067867,0.060351,0.049436,0.073468,0.054789,0.058979,0.088935,0.028985,0.049073,0.044907,0.052374,0.056514,0.049705,0.027058,0.065638,0.071953,0.039755,0.046988,0.066496,0.051579,0.059843,0.072234,0.037804,0.062369,0.053469,0.051053,0.040294,0.068393,0.024957,0.041997,0.027829,0.067077,0.034187,0.039675,0.046186,0.063958,0.051870
7-methylguanosine RNA capping (GO:0009452),0.386474,0.375981,0.371075,0.369548,0.381344,0.390262,0.382853,0.380089,0.384769,0.376531,0.376737,0.372272,0.388178,0.384520,0.383952,0.379058,0.376365,0.374223,0.389405,0.378269,0.368857,0.378652,0.367864,0.365541,0.382766,0.374305,0.379177,0.365543,0.376188,0.378731,0.374810,0.381255,0.368398,0.380297,0.394586,0.393355,0.387522,0.374710,0.382578,0.385260,...,0.390229,0.392053,0.381103,0.393404,0.386679,0.386918,0.397023,0.390504,0.386948,0.383626,0.394927,0.

In [22]:
list(pd.read_csv("gseapy_results/GO_Biological_Process_2018.human.enrichr.reports.txt", sep="\t").loc[:,"Overlap"])

['50/52',
 '40/41',
 '64/68',
 '44/46',
 '35/36',
 '24/24',
 '59/63',
 '34/35',
 '42/44',
 '212/240',
 '86/94',
 '57/61',
 '41/43',
 '64/69',
 '172/194',
 '55/59',
 '21/21',
 '62/67',
 '62/67',
 '253/289',
 '39/41',
 '88/97',
 '20/20',
 '20/20',
 '20/20',
 '20/20',
 '81/89',
 '94/104',
 '67/73',
 '29/30',
 '19/19',
 '19/19',
 '393/455',
 '28/29',
 '58/63',
 '58/63',
 '91/101',
 '199/227',
 '84/93',
 '18/18',
 '18/18',
 '18/18',
 '18/18',
 '27/28',
 '35/37',
 '17/17',
 '17/17',
 '17/17',
 '26/27',
 '26/27',
 '106/119',
 '124/140',
 '25/26',
 '25/26',
 '16/16',
 '67/74',
 '40/43',
 '47/51',
 '98/110',
 '197/226',
 '46/50',
 '24/25',
 '24/25',
 '24/25',
 '24/25',
 '59/65',
 '59/65',
 '15/15',
 '15/15',
 '15/15',
 '15/15',
 '15/15',
 '15/15',
 '15/15',
 '45/49',
 '38/41',
 '58/64',
 '58/64',
 '23/24',
 '23/24',
 '176/202',
 '64/71',
 '153/175',
 '30/32',
 '14/14',
 '14/14',
 '14/14',
 '14/14',
 '14/14',
 '14/14',
 '63/70',
 '99/112',
 '87/98',
 '22/23',
 '22/23',
 '22/23',
 '22/23',
 '43/4

In [27]:
GO=pd.read_csv("gseapy_results/GO_Biological_Process_2018.human.enrichr.reports.txt", sep="\t")

In [29]:
GO["size"]=[int(x.split("/")[0]) for x in list(GO["Overlap"])]

In [34]:
GO["ID"]=[x.split(":")[1][:-1] for x in list(GO["Term"])]

In [35]:
GO

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,size,ID
0,GO_Biological_Process_2018,B cell differentiation (GO:0030183),50/52,0.004227,0.999998,0,0,5.116553,2.796816e+01,IFNA5;ITGB1;IFNA4;IFNA7;IFNA6;IFNA1;FLT3;IFNA2...,50,0030183
1,GO_Biological_Process_2018,regulation of cellular metabolic process (GO:0...,40/41,0.004581,0.999998,0,0,8.183958,4.407826e+01,ANKRD13C;ARF4;ARF1;PANK2;PID1;RNF34;GBA;AGPAT1...,40,0031323
2,GO_Biological_Process_2018,negative regulation of phosphatase activity (G...,64/68,0.006298,0.999998,0,0,3.275431,1.659853e+01,PLEKHF2;TSKS;GSK3B;CSRNP2;CSRNP3;PLEKHF1;RBM26...,64,0010923
3,GO_Biological_Process_2018,cartilage development (GO:0051216),44/46,0.010338,0.999998,0,0,4.500936,2.057798e+01,WNT2B;COL11A2;SCX;PKD1;FGF2;FGF4;HOXA11;HYAL1;...,44,0051216
4,GO_Biological_Process_2018,regulation of potassium ion transport (GO:0043...,35/36,0.010351,0.999998,0,0,7.158803,3.272064e+01,DPP10;CD63;KCNE1;KCNE2;KCNE3;KCNE5;FHL1;KCNA5;...,35,0043266
...,...,...,...,...,...,...,...,...,...,...,...,...
5098,GO_Biological_Process_2018,ATP synthesis coupled proton transport (GO:001...,9/27,0.999998,0.999998,0,0,0.101596,2.308963e-07,STOML2;VPS9D1;ATP6V0A2;ATP5J;ATP6V0A4;TCIRG1;A...,9,0015986
5099,GO_Biological_Process_2018,purine ribonucleoside triphosphate biosyntheti...,8/27,0.999998,0.999998,0,0,0.085524,1.942754e-07,SURF1;TGFB1;IMPDH1;PRKAG2;ATP5J;ATP5H;ATP5G1;N...,8,0009206
5100,GO_Biological_Process_2018,mitochondrial ATP synthesis coupled proton tra...,5/21,0.999998,0.999998,0,0,0.063520,1.279207e-07,STOML2;ATP5J;ATP5H;CYC1;ATP5G1,5,0042776
5101,GO_Biological_Process_2018,septin ring organization (GO:0031106),5/17,0.999998,0.999998,0,0,0.084794,1.625812e-07,ANLN;RTKN;RTKN2;TMEM250;SEPT5,5,0031106


In [44]:
df

,101_19yr_Female_Caucasian,102_19yr_Male_Caucasian,103_19yr_Male_Caucasian,104_19yr_Male_Caucasian,105_20yr_Male_Caucasian,106_20yr_Female_Caucasian,107_31yr_Female_Caucasian,108_31_female_Caucasian,109_32_male_Unknown,110_32_female_BlackPuertoRican,111_33yr_Male_Caucasian,112_33yr_Male_Caucasian,114_37yr_Female_Unknown,115_37yr_Female_Unknown,117_37yr_Male_Caucasian,118_39yr_Male_Caucasian,120_51yr_Male_Caucasian,121_52yr_Male_Caucasian,122_55yr_Male_Caucasian,123_57_male_Caucasian,124_60yr_Male_Caucasian,125_71yr_Female_Caucasian,126_75_male_Caucasian,127_78yr_Male_Caucasian,128_80yr_Female_Caucasian,129_94_Male_Caucasian,130_89_Male_Caucasian,131_87_Male_Caucasian,132_90_Male_Caucasian,133_89_Male_Caucasian,134_86_Male_Caucasian,135_92_Male_Caucasian,136_87_Male_Caucasian,14_1yr_Male_Asian,15_12yr_Male_Caucasian,155_22yr_Male_Caucasian,156_25yr_Female_Caucasian,157_29yr_Male_Unknown,158_24YR_Male_Caucasian,159_23YR_Male_AmericanIndian,...,70_87_Male_caucasian,71_87_Male_caucasian,72_87_Male_Black,73_88_Male_Caucasian,74_89_Male_Caucasian,75_90_Male_Caucasian,76_91_Male_Caucasian,77_92_Female_Caucasian,78_96_Male_Caucasian,79_1_male_Caucasian,80_2_female_Caucasian,81_3_male_LatinoHispanic,82_3_male_Unknown,83_5yr_Male_Black,84_6yr_Male_Black,85_7yr_Male_Black,86_7yr_Male_Caucasian,87_8yr_Male_Caucasian,88_8yr_Male_Caucasian,89_9yr_Female_Black,90_10yr_Male_Black,91_10yr_Male_Caucasian,92_11_female_Caucasian,93_11_female_Caucasian,94_12_male_Caucasian,95_13yr_Male_WhiTEEASTINDIAN,97_16yr_Female_Unknown,98_17yr_Male_Caucasian,99_17yr_Male_Caucasian,139_8_Female-HGPS_Caucasian,141_8_Male-HGPS_Unknown,197_2yr3mos_F_Unknown,198_3yr9mos_F_Unknown,199_4yr8mos_F_Unknown,200_8yr6mos_M_Unknown,201_6ys11mos_F_Unknown,202_5yr0mos_F_Unknown,203_8yr10mos_M_Unknown,204_3yr0mos_F_Unknown,ID
Term|NES,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'de novo' posttranslational protein folding (GO:0051084),0.35089,0.347854,0.323341,0.319741,0.348308,0.353414,0.350498,0.360435,0.360693,0.334585,0.331541,0.350204,0.354583,0.326394,0.327429,0.344968,0.331023,0.345603,0.340241,0.338566,0.341722,0.316734,0.353803,0.345665,0.354654,0.347367,0.344354,0.352872,0.347037,0.352841,0.338798,0.357623,0.358612,0.335348,0.332923,0.34083,0.33846,0.35098,0.324259,0.329552,...,0.329551,0.330741,0.372936,0.319973,0.357746,0.393503,0.347189,0.3671,0.369298,0.335381,0.348068,0.343269,0.329249,0.337968,0.354243,0.352401,0.335017,0.32818,0.350953,0.342896,0.333524,0.348205,0.323334,0.321925,0.333707,0.344194,0.355421,0.327752,0.353976,0.339281,0.333005,0.351428,0.35369,0.335838,0.333286,0.345568,0.338217,0.351784,0.331427,0051084
3'-phosphoadenosine 5'-phosphosulfate metabolic process (GO:0050427),0.055188,0.0457785,0.0315279,0.0666365,0.0554838,0.0707949,0.0401815,0.0526637,0.0604488,0.0555353,0.0726799,0.0342725,0.0376347,0.0645476,0.053665,0.0757079,0.0576422,0.0473851,0.0552918,0.048208,0.0604545,0.0850343,0.0307755,0.0774771,0.0733505,0.0573718,0.0488117,0.0940535,0.0629418,0.0507391,0.0681348,0.0860503,0.0778614,0.0461281,0.049494,0.0450368,0.0638187,0.0491185,0.0729486,0.0719561,...,0.0556307,0.059293,0.0678666,0.0603509,0.0494361,0.0734684,0.0547887,0.058979,0.0889347,0.0289847,0.0490733,0.0449072,0.0523739,0.0565139,0.0497054,0.0270578,0.0656384,0.0719533,0.039755,0.0469883,0.0664955,0.0515787,0.0598433,0.0722337,0.0378038,0.0623686,0.0534685,0.0510527,0.0402938,0.0683926,0.0249575,0.041997,0.0278286,0.0670772,0.0341872,0.0396752,0.0461862,0.0639582,0.0518701,0050427
7-methylguanosine RNA capping (GO:0009452),0.386474,0.375981,0.371075,0.369548,0.381344,0.390262,0.382853,0.380089,0.384769,0.376531,0.376737,0.372272,0.388178,0.38452,0.383952,0.379058,0.376365,0.374223,0.389405,0.378269,0.368857,0.378652,0.367864,0.365541,0.382766,0.374305,0.379177,0.365543,0.376188,0.378731,0.37481,0.381255,0.368398,0.380297,0.394586,0.393355,0.387522,0.37471,0.382578,0.38526,...,0.392053,0.381103,0.393404,0.386679,0.386918,0.397023,0.390504,0

In [38]:
df=df.T

In [41]:
df["ID"]=[x.split(":")[1][:-1] for x in list(df.index)]

In [45]:
df=df.set_index("ID")

In [46]:
df

,101_19yr_Female_Caucasian,102_19yr_Male_Caucasian,103_19yr_Male_Caucasian,104_19yr_Male_Caucasian,105_20yr_Male_Caucasian,106_20yr_Female_Caucasian,107_31yr_Female_Caucasian,108_31_female_Caucasian,109_32_male_Unknown,110_32_female_BlackPuertoRican,111_33yr_Male_Caucasian,112_33yr_Male_Caucasian,114_37yr_Female_Unknown,115_37yr_Female_Unknown,117_37yr_Male_Caucasian,118_39yr_Male_Caucasian,120_51yr_Male_Caucasian,121_52yr_Male_Caucasian,122_55yr_Male_Caucasian,123_57_male_Caucasian,124_60yr_Male_Caucasian,125_71yr_Female_Caucasian,126_75_male_Caucasian,127_78yr_Male_Caucasian,128_80yr_Female_Caucasian,129_94_Male_Caucasian,130_89_Male_Caucasian,131_87_Male_Caucasian,132_90_Male_Caucasian,133_89_Male_Caucasian,134_86_Male_Caucasian,135_92_Male_Caucasian,136_87_Male_Caucasian,14_1yr_Male_Asian,15_12yr_Male_Caucasian,155_22yr_Male_Caucasian,156_25yr_Female_Caucasian,157_29yr_Male_Unknown,158_24YR_Male_Caucasian,159_23YR_Male_AmericanIndian,...,69_87_Female_caucasian,70_87_Male_caucasian,71_87_Male_caucasian,72_87_Male_Black,73_88_Male_Caucasian,74_89_Male_Caucasian,75_90_Male_Caucasian,76_91_Male_Caucasian,77_92_Female_Caucasian,78_96_Male_Caucasian,79_1_male_Caucasian,80_2_female_Caucasian,81_3_male_LatinoHispanic,82_3_male_Unknown,83_5yr_Male_Black,84_6yr_Male_Black,85_7yr_Male_Black,86_7yr_Male_Caucasian,87_8yr_Male_Caucasian,88_8yr_Male_Caucasian,89_9yr_Female_Black,90_10yr_Male_Black,91_10yr_Male_Caucasian,92_11_female_Caucasian,93_11_female_Caucasian,94_12_male_Caucasian,95_13yr_Male_WhiTEEASTINDIAN,97_16yr_Female_Unknown,98_17yr_Male_Caucasian,99_17yr_Male_Caucasian,139_8_Female-HGPS_Caucasian,141_8_Male-HGPS_Unknown,197_2yr3mos_F_Unknown,198_3yr9mos_F_Unknown,199_4yr8mos_F_Unknown,200_8yr6mos_M_Unknown,201_6ys11mos_F_Unknown,202_5yr0mos_F_Unknown,203_8yr10mos_M_Unknown,204_3yr0mos_F_Unknown
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0051084,0.35089,0.347854,0.323341,0.319741,0.348308,0.353414,0.350498,0.360435,0.360693,0.334585,0.331541,0.350204,0.354583,0.326394,0.327429,0.344968,0.331023,0.345603,0.340241,0.338566,0.341722,0.316734,0.353803,0.345665,0.354654,0.347367,0.344354,0.352872,0.347037,0.352841,0.338798,0.357623,0.358612,0.335348,0.332923,0.34083,0.33846,0.35098,0.324259,0.329552,...,0.334397,0.329551,0.330741,0.372936,0.319973,0.357746,0.393503,0.347189,0.3671,0.369298,0.335381,0.348068,0.343269,0.329249,0.337968,0.354243,0.352401,0.335017,0.32818,0.350953,0.342896,0.333524,0.348205,0.323334,0.321925,0.333707,0.344194,0.355421,0.327752,0.353976,0.339281,0.333005,0.351428,0.35369,0.335838,0.333286,0.345568,0.338217,0.351784,0.331427
0050427,0.055188,0.0457785,0.0315279,0.0666365,0.0554838,0.0707949,0.0401815,0.0526637,0.0604488,0.0555353,0.0726799,0.0342725,0.0376347,0.0645476,0.053665,0.0757079,0.0576422,0.0473851,0.0552918,0.048208,0.0604545,0.0850343,0.0307755,0.0774771,0.0733505,0.0573718,0.0488117,0.0940535,0.0629418,0.0507391,0.0681348,0.0860503,0.0778614,0.0461281,0.049494,0.0450368,0.0638187,0.0491185,0.0729486,0.0719561,...,0.0576306,0.0556307,0.059293,0.0678666,0.0603509,0.0494361,0.0734684,0.0547887,0.058979,0.0889347,0.0289847,0.0490733,0.0449072,0.0523739,0.0565139,0.0497054,0.0270578,0.0656384,0.0719533,0.039755,0.0469883,0.0664955,0.0515787,0.0598433,0.0722337,0.0378038,0.0623686,0.0534685,0.0510527,0.0402938,0.0683926,0.0249575,0.041997,0.0278286,0.0670772,0.0341872,0.0396752,0.0461862,0.0639582,0.0518701
0009452,0.386474,0.375981,0.371075,0.369548,0.381344,0.390262,0.382853,0.380089,0.384769,0.376531,0.376737,0.372272,0.388178,0.38452,0.383952,0.379058,0.376365,0.374223,0.389405,0.378269,0.368857,0.378652,0.367864,0.365541,0.382766,0.374305,0.379177,0.365543,0.376188,0.378731,0.37481,0.381255,0.368398,0.380297,0.394586,0.393355,0.387522,0.37471,0.382578,0.38526,...,0.390229,0.392053,0.381103,0.393404,0.386679,0.386918,0.397023,0.390504,0.386948,0.383626,0.394927,0.388429,0.389651,0.384742,0.403174,0.379645,0.395807,0.375853,0.375637,0.387032,0.400884,0.3

In [51]:
df.T[list(GO["ID"])].T

KeyError: ignored

In [56]:
GO

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,size,ID
0,GO_Biological_Process_2018,B cell differentiation (GO:0030183),50/52,0.004227,0.999998,0,0,5.116553,2.796816e+01,IFNA5;ITGB1;IFNA4;IFNA7;IFNA6;IFNA1;FLT3;IFNA2...,50,0030183
1,GO_Biological_Process_2018,regulation of cellular metabolic process (GO:0...,40/41,0.004581,0.999998,0,0,8.183958,4.407826e+01,ANKRD13C;ARF4;ARF1;PANK2;PID1;RNF34;GBA;AGPAT1...,40,0031323
2,GO_Biological_Process_2018,negative regulation of phosphatase activity (G...,64/68,0.006298,0.999998,0,0,3.275431,1.659853e+01,PLEKHF2;TSKS;GSK3B;CSRNP2;CSRNP3;PLEKHF1;RBM26...,64,0010923
3,GO_Biological_Process_2018,cartilage development (GO:0051216),44/46,0.010338,0.999998,0,0,4.500936,2.057798e+01,WNT2B;COL11A2;SCX;PKD1;FGF2;FGF4;HOXA11;HYAL1;...,44,0051216
4,GO_Biological_Process_2018,regulation of potassium ion transport (GO:0043...,35/36,0.010351,0.999998,0,0,7.158803,3.272064e+01,DPP10;CD63;KCNE1;KCNE2;KCNE3;KCNE5;FHL1;KCNA5;...,35,0043266
...,...,...,...,...,...,...,...,...,...,...,...,...
5098,GO_Biological_Process_2018,ATP synthesis coupled proton transport (GO:001...,9/27,0.999998,0.999998,0,0,0.101596,2.308963e-07,STOML2;VPS9D1;ATP6V0A2;ATP5J;ATP6V0A4;TCIRG1;A...,9,0015986
5099,GO_Biological_Process_2018,purine ribonucleoside triphosphate biosyntheti...,8/27,0.999998,0.999998,0,0,0.085524,1.942754e-07,SURF1;TGFB1;IMPDH1;PRKAG2;ATP5J;ATP5H;ATP5G1;N...,8,0009206
5100,GO_Biological_Process_2018,mitochondrial ATP synthesis coupled proton tra...,5/21,0.999998,0.999998,0,0,0.063520,1.279207e-07,STOML2;ATP5J;ATP5H;CYC1;ATP5G1,5,0042776
5101,GO_Biological_Process_2018,septin ring organization (GO:0031106),5/17,0.999998,0.999998,0,0,0.084794,1.625812e-07,ANLN;RTKN;RTKN2;TMEM250;SEPT5,5,0031106


In [65]:
GO1=GO[GO["size"]>20]
GO1=GO1[GO1["size"]<500]

In [66]:
GO1

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,size,ID
0,GO_Biological_Process_2018,B cell differentiation (GO:0030183),50/52,0.004227,0.999998,0,0,5.116553,2.796816e+01,IFNA5;ITGB1;IFNA4;IFNA7;IFNA6;IFNA1;FLT3;IFNA2...,50,0030183
1,GO_Biological_Process_2018,regulation of cellular metabolic process (GO:0...,40/41,0.004581,0.999998,0,0,8.183958,4.407826e+01,ANKRD13C;ARF4;ARF1;PANK2;PID1;RNF34;GBA;AGPAT1...,40,0031323
2,GO_Biological_Process_2018,negative regulation of phosphatase activity (G...,64/68,0.006298,0.999998,0,0,3.275431,1.659853e+01,PLEKHF2;TSKS;GSK3B;CSRNP2;CSRNP3;PLEKHF1;RBM26...,64,0010923
3,GO_Biological_Process_2018,cartilage development (GO:0051216),44/46,0.010338,0.999998,0,0,4.500936,2.057798e+01,WNT2B;COL11A2;SCX;PKD1;FGF2;FGF4;HOXA11;HYAL1;...,44,0051216
4,GO_Biological_Process_2018,regulation of potassium ion transport (GO:0043...,35/36,0.010351,0.999998,0,0,7.158803,3.272064e+01,DPP10;CD63;KCNE1;KCNE2;KCNE3;KCNE5;FHL1;KCNA5;...,35,0043266
...,...,...,...,...,...,...,...,...,...,...,...,...
5088,GO_Biological_Process_2018,plasma membrane invagination (GO:0099024),23/103,0.999996,0.999998,0,0,0.057393,2.205849e-07,SCARB2;MSR1;ITGAM;SNX33;ARHGAP25;AIF1;ARHGAP12...,23,0099024
5090,GO_Biological_Process_2018,purine ribonucleotide biosynthetic process (GO...,42/75,0.999997,0.999998,0,0,0.257972,8.890530e-07,GUCY2C;SLC35B2;PANK2;SLC26A2;GUCY2D;SLC26A1;NP...,42,0009152
5091,GO_Biological_Process_2018,chromatin assembly (GO:0031497),34/65,0.999997,0.999998,0,0,0.222333,7.253992e-07,HIST1H2BN;HIST1H2BJ;HIST1H2BK;HMGB2;RSF1;CTCF;...,34,0031497
5092,GO_Biological_Process_2018,nucleosome assembly (GO:0006334),30/60,0.999997,0.999998,0,0,0.202726,6.416098e-07,HIST1H2BN;HIST1H2BJ;HIST1H2BK;HMGB2;RSF1;SPTY2...,30,0006334


In [64]:
len(set(GO1["ID"]) & set(df.index))

2273

In [59]:
df

,101_19yr_Female_Caucasian,102_19yr_Male_Caucasian,103_19yr_Male_Caucasian,104_19yr_Male_Caucasian,105_20yr_Male_Caucasian,106_20yr_Female_Caucasian,107_31yr_Female_Caucasian,108_31_female_Caucasian,109_32_male_Unknown,110_32_female_BlackPuertoRican,111_33yr_Male_Caucasian,112_33yr_Male_Caucasian,114_37yr_Female_Unknown,115_37yr_Female_Unknown,117_37yr_Male_Caucasian,118_39yr_Male_Caucasian,120_51yr_Male_Caucasian,121_52yr_Male_Caucasian,122_55yr_Male_Caucasian,123_57_male_Caucasian,124_60yr_Male_Caucasian,125_71yr_Female_Caucasian,126_75_male_Caucasian,127_78yr_Male_Caucasian,128_80yr_Female_Caucasian,129_94_Male_Caucasian,130_89_Male_Caucasian,131_87_Male_Caucasian,132_90_Male_Caucasian,133_89_Male_Caucasian,134_86_Male_Caucasian,135_92_Male_Caucasian,136_87_Male_Caucasian,14_1yr_Male_Asian,15_12yr_Male_Caucasian,155_22yr_Male_Caucasian,156_25yr_Female_Caucasian,157_29yr_Male_Unknown,158_24YR_Male_Caucasian,159_23YR_Male_AmericanIndian,...,69_87_Female_caucasian,70_87_Male_caucasian,71_87_Male_caucasian,72_87_Male_Black,73_88_Male_Caucasian,74_89_Male_Caucasian,75_90_Male_Caucasian,76_91_Male_Caucasian,77_92_Female_Caucasian,78_96_Male_Caucasian,79_1_male_Caucasian,80_2_female_Caucasian,81_3_male_LatinoHispanic,82_3_male_Unknown,83_5yr_Male_Black,84_6yr_Male_Black,85_7yr_Male_Black,86_7yr_Male_Caucasian,87_8yr_Male_Caucasian,88_8yr_Male_Caucasian,89_9yr_Female_Black,90_10yr_Male_Black,91_10yr_Male_Caucasian,92_11_female_Caucasian,93_11_female_Caucasian,94_12_male_Caucasian,95_13yr_Male_WhiTEEASTINDIAN,97_16yr_Female_Unknown,98_17yr_Male_Caucasian,99_17yr_Male_Caucasian,139_8_Female-HGPS_Caucasian,141_8_Male-HGPS_Unknown,197_2yr3mos_F_Unknown,198_3yr9mos_F_Unknown,199_4yr8mos_F_Unknown,200_8yr6mos_M_Unknown,201_6ys11mos_F_Unknown,202_5yr0mos_F_Unknown,203_8yr10mos_M_Unknown,204_3yr0mos_F_Unknown
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0051084,0.35089,0.347854,0.323341,0.319741,0.348308,0.353414,0.350498,0.360435,0.360693,0.334585,0.331541,0.350204,0.354583,0.326394,0.327429,0.344968,0.331023,0.345603,0.340241,0.338566,0.341722,0.316734,0.353803,0.345665,0.354654,0.347367,0.344354,0.352872,0.347037,0.352841,0.338798,0.357623,0.358612,0.335348,0.332923,0.34083,0.33846,0.35098,0.324259,0.329552,...,0.334397,0.329551,0.330741,0.372936,0.319973,0.357746,0.393503,0.347189,0.3671,0.369298,0.335381,0.348068,0.343269,0.329249,0.337968,0.354243,0.352401,0.335017,0.32818,0.350953,0.342896,0.333524,0.348205,0.323334,0.321925,0.333707,0.344194,0.355421,0.327752,0.353976,0.339281,0.333005,0.351428,0.35369,0.335838,0.333286,0.345568,0.338217,0.351784,0.331427
0050427,0.055188,0.0457785,0.0315279,0.0666365,0.0554838,0.0707949,0.0401815,0.0526637,0.0604488,0.0555353,0.0726799,0.0342725,0.0376347,0.0645476,0.053665,0.0757079,0.0576422,0.0473851,0.0552918,0.048208,0.0604545,0.0850343,0.0307755,0.0774771,0.0733505,0.0573718,0.0488117,0.0940535,0.0629418,0.0507391,0.0681348,0.0860503,0.0778614,0.0461281,0.049494,0.0450368,0.0638187,0.0491185,0.0729486,0.0719561,...,0.0576306,0.0556307,0.059293,0.0678666,0.0603509,0.0494361,0.0734684,0.0547887,0.058979,0.0889347,0.0289847,0.0490733,0.0449072,0.0523739,0.0565139,0.0497054,0.0270578,0.0656384,0.0719533,0.039755,0.0469883,0.0664955,0.0515787,0.0598433,0.0722337,0.0378038,0.0623686,0.0534685,0.0510527,0.0402938,0.0683926,0.0249575,0.041997,0.0278286,0.0670772,0.0341872,0.0396752,0.0461862,0.0639582,0.0518701
0009452,0.386474,0.375981,0.371075,0.369548,0.381344,0.390262,0.382853,0.380089,0.384769,0.376531,0.376737,0.372272,0.388178,0.38452,0.383952,0.379058,0.376365,0.374223,0.389405,0.378269,0.368857,0.378652,0.367864,0.365541,0.382766,0.374305,0.379177,0.365543,0.376188,0.378731,0.37481,0.381255,0.368398,0.380297,0.394586,0.393355,0.387522,0.37471,0.382578,0.38526,...,0.390229,0.392053,0.381103,0.393404,0.386679,0.386918,0.397023,0.390504,0.386948,0.383626,0.394927,0.388429,0.389651,0.384742,0.403174,0.379645,0.395807,0.375853,0.375637,0.387032,0.400884,0.3